# Data Mining – Cleaning

---

In [1]:
import pandas as pd
import numpy as np
import datetime
import re

In [2]:
df = pd.read_csv("data/scraping_full.csv")

In [57]:
df.head(2)

,url,chanel,title,nb_images,author_name,cleaned_article,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/ap-samsung-spon...,Business,AP's Twitter to Begin Displaying Sponsored Tweets,1,Seth Fiegerman,The Associated Press is the latest news organ...,731.0,9.0,255.0,0.604743,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.0,0.0,0.5,0.0,711
1,http://mashable.com/2013/01/07/apple-40-billio...,Business,Apple's App Store Passes 40 Billion Downloads,1,Seth Fiegerman,It looks like 2012 was a pretty good year for...,731.0,9.0,211.0,0.575130,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.0,0.0,0.5,0.0,1500


Le jeu de données possède une analyse en NLP (Natural Langage Processing), toutefois pour notre rapport nous allons en refaire une et comparer ensuite nos résultats avec celui du jeu de données initial.

### Choix des variables à inclure

On récupère les colonnes intéréssantes pour notre analyse (celles qui font partie du site internet et ne relèvent pas de l'analyse NLP).

In [21]:
list(df.columns)

['url',
 'chanel',
 'title',
 'nb_images',
 'author_name',
 'cleaned_article',
 ' timedelta',
 ' n_tokens_title',
 ' n_tokens_content',
 ' n_unique_tokens',
 ' n_non_stop_words',
 ' n_non_stop_unique_tokens',
 ' num_hrefs',
 ' num_self_hrefs',
 ' num_imgs',
 ' num_videos',
 ' average_token_length',
 ' num_keywords',
 ' data_channel_is_lifestyle',
 ' data_channel_is_entertainment',
 ' data_channel_is_bus',
 ' data_channel_is_socmed',
 ' data_channel_is_tech',
 ' data_channel_is_world',
 ' kw_min_min',
 ' kw_max_min',
 ' kw_avg_min',
 ' kw_min_max',
 ' kw_max_max',
 ' kw_avg_max',
 ' kw_min_avg',
 ' kw_max_avg',
 ' kw_avg_avg',
 ' self_reference_min_shares',
 ' self_reference_max_shares',
 ' self_reference_avg_sharess',
 ' weekday_is_monday',
 ' weekday_is_tuesday',
 ' weekday_is_wednesday',
 ' weekday_is_thursday',
 ' weekday_is_friday',
 ' weekday_is_saturday',
 ' weekday_is_sunday',
 ' is_weekend',
 ' LDA_00',
 ' LDA_01',
 ' LDA_02',
 ' LDA_03',
 ' LDA_04',
 ' global_subjectivity',
 '

On récupère donc les variables : `shares`, `titre`, `content`, `num_img`, `num_video`, `num_hrefs`, `timedelta`

In [21]:
df_new = df.loc[:,["url", " shares", 'cleaned_article','author_name',
                   'title', "nb_images",' num_videos', 
                   ' num_hrefs',' timedelta',' num_keywords','chanel']]

In [23]:
df_new.head(4)

,url,shares,cleaned_article,author_name,title,nb_images,num_videos,num_hrefs,timedelta,num_keywords,chanel
0,http://mashable.com/2013/01/07/ap-samsung-spon...,711,The Associated Press is the latest news organ...,Seth Fiegerman,AP's Twitter to Begin Displaying Sponsored Tweets,1,0.0,3.0,731.0,4.0,Business
1,http://mashable.com/2013/01/07/apple-40-billio...,1500,It looks like 2012 was a pretty good year for...,Seth Fiegerman,Apple's App Store Passes 40 Billion Downloads,1,0.0,3.0,731.0,6.0,Business
2,http://mashable.com/2013/01/07/astronaut-notre...,1200,"When it comes to college football, NASA astro...",Tariq Malik,This Astronaut Is Rooting for Notre Dame Tonight,3,0.0,9.0,731.0,7.0,Entertainment
3,http://mashable.com/2013/01/07/att-u-verse-apps/,505,LAS VEGAS — Sharing photos and videos on your...,Emily Price,New U-Verse Apps Simplify Sharing Photos and V...,20,0.0,19.0,731.0,7.0,Tech


### Gestion des valeurs manquantes

Les valeurs manquantes étaient en `string`, on les remet en None

In [28]:
df_new = df_new.replace({'None': None})

Le nombre de valeurs manquantes pour chaque variable

In [56]:
df_new.isnull().sum()

url                0
 shares            0
cleaned_article    0
author_name        0
title              0
nb_images          0
 num_videos        0
 num_hrefs         0
 num_keywords      0
chanel             0
date               0
day                0
week               0
dtype: int64

On décide de retirer les observations où il existe des valeurs manquantes. Après une recherche de leur raison on peut en définir plusieurs types : 
- `The Bad News` signifie que le lien est mort.
- Lorsque le titre est collecté mais pas le contenu, c'est que le site ne contient pas de texte dans l'article, ou seulement des images/vidéos.
- Lorsque le contenu et le titre est manquant, c'est que la structure de l'article est différente et que le scraping n'a pas pu collecter l'information, ou qu'il s'agit d'un article de nature différente (galerie de photos par exemple).

In [30]:
df_new.dropna(inplace=True)

On se retrouve avec maintenant 38 405 observations, soit 193 lignes en moins. Cela équivaut à 3% de NA (cela est donc négligeable).

In [34]:
df_new.shape[0] - df.shape[0]

-1238

In [31]:
str(np.round(1 - df_new.shape[0] / df.shape[0],3)*100) + '%'

'3.1%'

### Observations du jeu de données

On regarde les observations de notre nouveau DataFrame : 

In [28]:
df_new.nb_images = df_new.nb_images.astype(int)

In [29]:
pd.options.display.float_format = '{:.0f}'.format

df_new.describe()

,shares,nb_images,num_videos,num_hrefs,timedelta,num_keywords
count,38405,38405,38405,38405,38405,38405
mean,3375,5,1,11,354,7
std,11253,8,4,11,214,2
min,5,0,0,0,8,1
25%,946,1,0,4,165,6
50%,1400,2,0,7,338,7
75%,2800,6,1,14,542,9
max,843300,128,91,304,731,10


On peut voir que le nombre de partage varie entre 1 et 843 300. Pour le cas des dates, elles sont affichés en nombre de jours de différence par rapport à la date d'obtention du jeu de données. On peut donc les transformer en date dès que l'on sait la date d'un article.

### Parsing des dates

On peut remarquer que la date de l'article est inclut dans le lien. Pour éviter les erreurs de dates en les calculant par rapport à une autre (ce qui était le cas après vérification), on peut donc directement récuperer celle-ci à partir d'une regex. On peut donc obtenir le jour de la semaine (Lundi, mardi, etc.) et inclure une variable binaire s'il s'agit d'un jour en semaine, ou du week-end.

In [42]:
def find_date(url):
    regex = re.findall('.com/([/0-9]*)/',url)[0]
    date = pd.to_datetime(regex)
    day = date.day_name()
    if day == "Saturday" or day == "Sunday" :
        week = 1
    else : 
        week = 0
    return pd.Series([date,day,week]) 

In [43]:
df_new[["date",'day','week']] = df_new.url.apply(lambda x : find_date(x))

In [44]:
df_new = df_new.drop(columns=[" timedelta"])

In [55]:
df_new.tail(2)

,url,shares,cleaned_article,author_name,title,nb_images,num_videos,num_hrefs,num_keywords,chanel,date,day,week
39641,http://mashable.com/2014/12/27/ukraine-blasts/,1100,Ukrainians were on high alert on Saturday aft...,Christopher Miller,Two dead after Ukraine rocked by series of blasts,2,0.0,10.0,5.0,World,2014-12-27,Saturday,1
39642,http://mashable.com/2014/12/27/youtube-channel...,1300,We collectively watch more than 6 billion hou...,Yohana Desta,8 YouTube channels to watch in 2015,1,2.0,1.0,4.0,Entertainment,2014-12-27,Saturday,1


On enregistre le csv des données.

In [17]:
df_new.to_csv("data/data_cleaned.csv")